In [ ]:
# default_exp core

## torchtools

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def test():
    '''a test function'''
    print('test')

In [ ]:
test()

test


### Section 2